In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_excel('esg_data.xlsx', sheet_name = "Data", engine='openpyxl') #in python 3.9 xlsx files are deprecated, so we will use openpyxl

df.head()

In [ ]:
#mapping each country to a continent and an income group

metadata = pd.read_excel('World_Metadata.xlsx', sheet_name= "Country - Metadata", engine= "openpyxl")

metadata = metadata[['Code', 'Income Group', 'Region']]

merged = df.merge(metadata, how = 'left', left_on = 'CountryCode', right_on = 'Code')
merged.head()



In [ ]:
#unpivoting the original dataframe to collapse the years into 1 column and get rid of useless columns

unpivot_df = merged.melt(id_vars = ["Country", "CountryCode", "Category", "SeriesCode",'Code', 'Income Group', 'Region'], value_name = "Value", var_name = 'Year') 
unpivot_df = unpivot_df[unpivot_df['Value'] != ".."]  #removing .. rows
unpivot_df.drop(['CountryCode', 'SeriesCode'], axis = 1, inplace = True)  #dropping the useless columns
unpivot_df = unpivot_df[~unpivot_df['Income Group'].isna()] # clear out NA income groups as these were just custom categories
unpivot_df.head()

In [ ]:
def check_lenght(dataframe):
    for i in range(2011, 2021):
        current_year = f"{i}"
        year_data = dataframe[dataframe.Year == int(current_year)]
        print(current_year + " has " + str(len(year_data)) + " rows")

check_lenght(unpivot_df)

# a good tradeoff will be to choose the year 2018 as the year for our current analysis, due to the high 2019 and 2020 having a low nr of rows


In [ ]:
#next, to be able to analyse the data in a more code-efficient way, we will repivot the data

dataframe_2018 = unpivot_df[unpivot_df.Year == 2018]
new_df = dataframe_2018.pivot(index = ['Country', 'Income Group','Region'], columns = 'Category', values = 'Value') #pivot the data
new_df = new_df.fillna(new_df.mean()) #fill N/A values with the mean

new_df.head()

In [ ]:
def correlation_finder(df, threshold):
    coeff = df.corr(method = 'pearson') #pearson correlation of all variables

    #find the combinations of variables that have their pearson correlation coefficient > threshhold or < -threshhold
    correlations = coeff.where((coeff< -threshold) | (coeff > threshold) & (coeff !=1)).stack() 
    return correlations 

corr = correlation_finder(new_df, 0.5)
corr.filter(like = 'CO2 emissions (metric tons per capita)') #check the variables highly correlated with CO2 emissions



In [ ]:
keep_variables = ['CO2 emissions (metric tons per capita)','Agriculture, forestry, and fishing, value added (% of GDP)', 'Government Effectiveness: Estimate', 'Individuals using the Internet (% of population)','Life expectancy at birth, total (years)', 'Renewable energy consumption (% of total final energy consumption)']

final_df = new_df.filter(keep_variables, axis=1)
final_df.rename(columns = {"Agriculture, forestry, and fishing, value added (% of GDP)" : "AFF" ,"CO2 emissions (metric tons per capita)" : "CO2 Emissions", 
                            "Government Effectiveness: Estimate": "Government Effectiveness", "Individuals using the Internet (% of population)" : "Individuals using the Internet", 
                            "Life expectancy at birth, total (years)": "Life Expectancy", "Renewable energy consumption (% of total final energy consumption)": "Renewable Energy Consumption"}, 
                            inplace=True)
final_df.reset_index(inplace=True)
final_df.head()



In [ ]:
final_df.to_csv('Final_Dataset.csv')

In [ ]:
# #Transform categorical variable "Income Group" into numerical

# income_categories = pd.unique(small_df['Income Group'])
# income_categories

# incomes = []

# for i in small_df['Income Group']:
#     if i == "Low income":
#         incomes.append(1)
#     elif i == "Lower middle income":
#         incomes.append(2)
#     elif i == "Upper middle income":
#         incomes.append(3)
#     else:
#         incomes.append(4)

# small_df['Income Group'] = incomes
